In [51]:
!pip install holidays -q
!pip install workalendar -q
#!pip install fasttext -q

In [2]:
%%capture
import catboost
import pandas as pd
import numpy as np
import fasttext
import fasttext.util
from tqdm import tqdm
#fasttext.util.download_model("ru")
#fasttext.util.reduce_model(ft_model, 100)


In [3]:
file_name = r"../TenderHack_Москва_train_data.xlsx" 
columns = ['id', 'status', 'name', 'okpd2','kpgz','region','nmck','price','date','participans','bet','inn']

spgz_name = "../spgz_kpgz.xlsx"
df_kpgz = pd.read_excel(spgz_name)
df_kpgz = df_kpgz['КПГЗ']

df = pd.read_excel(file_name, header=None, names=columns, skiprows=1)
df['okpd2'] = df['okpd2'].fillna(0)
prod_df = df
prod_df = prod_df.drop('id', axis='columns')
prod_df.dtypes

status          object
name            object
okpd2           object
kpgz            object
region          object
nmck           float64
price          float64
date            object
participans      int64
bet              int64
inn             object
dtype: object

In [4]:
prod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246762 entries, 0 to 246761
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   status       246762 non-null  object 
 1   name         246762 non-null  object 
 2   okpd2        246762 non-null  object 
 3   kpgz         224212 non-null  object 
 4   region       246762 non-null  object 
 5   nmck         246762 non-null  float64
 6   price        246762 non-null  float64
 7   date         246762 non-null  object 
 8   participans  246762 non-null  int64  
 9   bet          246762 non-null  int64  
 10  inn          246762 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 20.7+ MB


In [5]:
prod_df.head()

,status,name,okpd2,kpgz,region,nmck,price,date,participans,bet,inn
0,Завершена,СТУЛЬЯ УЧЕНИЧЕСКИЕ,0,01.06.01.03.01;01.06.01.03.01,Москва,596790.0,593806.05,2021-06-30 11:20:05.720,1,1,d8912494ba2edd3bfeba55206a78a0f5
1,Не состоялась,МЕДИЦИНСКИЕ РАСХОДНЫЕ МАТЕРИАЛЫ,0,01.02.10.50.33;01.02.10.43.05.01,Москва,4964.9,0.00,2022-10-04 09:16:04.770,0,0,237a5c57a66f02f8deb152e21f33863a
2,Завершена,Видеокарта Palit PCI-E,0,01.13.17.08,Москва,462000.0,311850.00,2021-07-01 13:23:09.177,6,65,040f1570117a744d529f4675f827a50f
3,Завершена,"ТОВАРЫ ИНФОРМАЦИОННО-ТЕХНОЛОГИЧЕСКИЕ, СРЕДСТВА...",0,01.13.11.03.02;01.13.04.04.06.05;01.10.04.04.0...,Москва,505555.0,460042.90,2021-03-16 10:42:20.810,6,18,c00c03dca0274fe43fc34e974434a927
4,Завершена,Пинцет острый,0,01.02.10.06.48.04,Москва,2000.0,1980.00,2022-05-26 11:04:42.597,2,2,cc997efa7a6742b6119a3c253a084e80


In [6]:
"""
Завершена
Не состоялась
Снята с публикации
Активна"""

prod_df["status"].value_counts()
cleanup_nums = {"status": {"Завершена": 1, "Не состоялась": 0, 'Снята с публикации': 0, 'Активна': -1}}
prod_df = prod_df.replace(cleanup_nums)

prod_df.head()

print(prod_df["status"].unique())

[ 1  0 -1]


In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for cat_var in ['inn']:
    prod_df[cat_var] = le.fit_transform(prod_df[cat_var])  
    
print(prod_df["inn"].unique())
print(prod_df["inn"].max())

[2808  443   56 ... 2689 1099  492]
3334


In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for cat_var in ['region']:
    prod_df[cat_var] = le.fit_transform(prod_df[cat_var])  
    
print(prod_df["region"].unique())
print(prod_df["region"].max())

[2 4 5 7 9 6 1 0 3 8]
9


In [9]:
def get_names_embeddings(kpgzs: str, mapping: dict, ft_model) -> str:
    try:
        kpgzs = kpgzs.strip()
        result = ""
        for kpgz in kpgzs.split(";"):
            result+=" "+mapping[kpgz]
        return ft_model.get_sentence_vector(result.strip())
    except Exception as e:
        return []

ft_model = fasttext.load_model("../cc.ru.100.bin")
mapping = {}
for kpgz, name in zip(df_kpgz.str.split().str[0], df_kpgz.str.split().str[1:].str.join(" ")):
    mapping[kpgz] = name

prod_df["kpgz"] = prod_df.kpgz.map(lambda x:get_names_embeddings(x, mapping=mapping, ft_model=ft_model))


# prod_df = prod_df.drop('name', axis='columns')
# prod_df = prod_df.drop('okpd2', axis='columns')
# prod_df = prod_df.drop('kpgz', axis='columns')

In [45]:
vecs = pd.DataFrame([list(x) for x in prod_df.kpgz.to_numpy()])

In [48]:
prod_df = pd.concat([prod_df, vecs], axis=1)
prod_df = prod_df.drop('kpgz', axis = 1)

In [49]:
prod_df

,status,name,okpd2,region,nmck,price,date,participans,bet,inn,...,90,91,92,93,94,95,96,97,98,99
0,1,СТУЛЬЯ УЧЕНИЧЕСКИЕ,0,2,596790.0,593806.05,2021-06-30 11:20:05.720,1,1,2808,...,-0.030289,-0.013461,0.082521,0.031594,-0.051453,-0.030685,0.030317,0.120106,0.062317,-0.026281
1,0,МЕДИЦИНСКИЕ РАСХОДНЫЕ МАТЕРИАЛЫ,0,2,4964.9,0.00,2022-10-04 09:16:04.770,0,0,443,...,0.077336,0.022978,0.047081,0.043159,-0.079364,-0.055020,0.075617,0.112388,0.020363,0.000546
2,1,Видеокарта Palit PCI-E,0,2,462000.0,311850.00,2021-07-01 13:23:09.177,6,65,56,...,-0.005300,-0.064341,-0.013175,0.109792,-0.000968,-0.047890,0.038708,0.180994,0.044681,-0.083619
3,1,"ТОВАРЫ ИНФОРМАЦИОННО-ТЕХНОЛОГИЧЕСКИЕ, СРЕДСТВА...",0,2,505555.0,460042.90,2021-03-16 10:42:20.810,6,18,2476,...,0.037284,-0.069748,-0.033464,-0.003649,0.006201,-0.046723,-0.002788,0.138808,0.053101,-0.055684
4,1,Пинцет острый,0,2,2000.0,1980.00,2022-05-26 11:04:42.597,2,2,2639,...,0.030939,-0.028657,0.031010,0.020796,-0.040028,-0.093241,0.078003,0.022769,0.143553,-0.060260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246757,0,Червячный хомут FORTISFLEX 110-130,0,2,1000.0,0.00,2022-03-30 12:23:15.930,0,0,2484,...,0.047160,-0.028300,0.002572,0.039058,-0.033461,0.018643,0.077797,0.150354,0.039075,-0.074555
246758,0,"Маскировочная сеть, ""Стандарт"", МС1-6, 3х6 м",0,2,60000.0,0.00,2022-10-03 09:42:05.507,0,0,2730,...,0.033528,-0.146428,-0.009694,0.031503,-0.037316,-0.081252,0.052857,0.126392,0.066388,-0.057452
246759,0,"ТРАНСПОРТНЫЕ СРЕДСТВА, РАСХОДНЫЕ МАТЕРИАЛЫ И З...",0,2,402582.6,0.00,2021-04-20 09:13:12.317,0,0,1261,...,0.036883,-0.067444,-0.025752,0.005859,-0.037086,-0.018055,0.043388,0.155591,0.031289,-0.093758
246760,1,БЛАНКИ,0,2,18600.0,10500.00,2022-10-28 11:47:01.503,5,9,2842,...,0.079016,0.013745,-0.018415,-0.040252,-0.012082,-0.081684,0.078668,0.133580,0.094387,-0.042714


In [52]:
prod_df['date'] = pd.to_datetime(prod_df.date, format='%Y-%m-%d %H:%M:%S')
import holidays
from workalendar.europe import Russia

range_of_dates = pd.date_range("2022-01-01", "2023-12-31")
holidays = holidays.Russia(years=[2022, 2023]).items()
hdays = []
for day in holidays:
    hdays.append(day[0])

def season_from_date(date):
    year = str(date.year)
    seasons = {'spring': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'summer': pd.date_range(start='21/06/'+year, end='22/09/'+year),
               'autumn': pd.date_range(start='23/09/'+year, end='20/12/'+year)}
    if date in seasons['spring']:
        return '1'
    if date in seasons['summer']:
        return '2'
    if date in seasons['autumn']:
        return '3'
    else:
        return '4'

df_dt = pd.DataFrame()
df_dt['date_year'] = prod_df['date'].dt.year
df_dt['date_month'] = prod_df['date'].dt.month
df_dt['date_week'] = prod_df['date'].dt.isocalendar().week 
df_dt["date_weekend"] = df_dt["date_week"] >= 5
df_dt["date_weekend"] = df_dt["date_weekend"].astype(int)
df_dt['date_day'] = prod_df['date'].dt.day
df_dt['date_hour'] = prod_df['date'].dt.hour
df_dt['date_minute'] = prod_df['date'].dt.minute
df_dt['date_dayofweek'] = prod_df['date'].dt.dayofweek
df_dt['date_quarter'] = prod_df['date'].dt.quarter
#df_dt["delta_to_holiday"] = df['date'].isin(hdays)
df_dt["is_holiday"] = prod_df['date'].isin(hdays)
df_dt["is_holiday"] = df_dt["is_holiday"].astype(int)
df_dt['date_season'] = prod_df.date.map(season_from_date)
prod_df = pd.concat([prod_df, df_dt], axis=1)
prod_df = prod_df.drop('date', axis='columns')
prod_df

/home/chaos_ruler/.venv/lib/python3.10/site-packages/pandas/core/base.py:880: UserWarning: Parsing '21/03/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  new_values = map_f(values, mapper)
/home/chaos_ruler/.venv/lib/python3.10/site-packages/pandas/core/base.py:880: UserWarning: Parsing '20/06/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  new_values = map_f(values, mapper)
/home/chaos_ruler/.venv/lib/python3.10/site-packages/pandas/core/base.py:880: UserWarning: Parsing '21/06/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  new_values = map_f(values, mapper)
/home/chaos_ruler/.venv/lib/python3.10/site-packages/pandas/core/base.py:880: UserWarning: Parsing '22/09/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  new_values = map_f(values, mapper)
/home/ch

,status,name,okpd2,region,nmck,price,participans,bet,inn,0,...,date_month,date_week,date_weekend,date_day,date_hour,date_minute,date_dayofweek,date_quarter,is_holiday,date_season
0,1,СТУЛЬЯ УЧЕНИЧЕСКИЕ,0,2,596790.0,593806.05,1,1,2808,0.161288,...,6,26,1,30,11,20,2,2,0,4
1,0,МЕДИЦИНСКИЕ РАСХОДНЫЕ МАТЕРИАЛЫ,0,2,4964.9,0.00,0,0,443,0.191935,...,10,40,1,4,9,16,1,4,0,4
2,1,Видеокарта Palit PCI-E,0,2,462000.0,311850.00,6,65,56,0.221634,...,7,26,1,1,13,23,3,3,0,4
3,1,"ТОВАРЫ ИНФОРМАЦИОННО-ТЕХНОЛОГИЧЕСКИЕ, СРЕДСТВА...",0,2,505555.0,460042.90,6,18,2476,0.159812,...,3,11,1,16,10,42,1,1,0,4
4,1,Пинцет острый,0,2,2000.0,1980.00,2,2,2639,0.144853,...,5,21,1,26,11,4,3,2,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246757,0,Червячный хомут FORTISFLEX 110-130,0,2,1000.0,0.00,0,0,2484,0.192599,...,3,13,1,30,12,23,2,1,0,4
246758,0,"Маскировочная сеть, ""Стандарт"", МС1-6, 3х6 м",0,2,60000.0,0.00,0,0,2730,0.280649,...,10,40,1,3,9,42,0,4,0,4
246759,0,"ТРАНСПОРТНЫЕ СРЕДСТВА, РАСХОДНЫЕ МАТЕРИАЛЫ И З...",0,2,402582.6,0.00,0,0,1261,0.194159,...,4,16,1,20,9,13,1,2,0,4
246760,1,БЛАНКИ,0,2,18600.0,10500.00,5,9,2842,0.332767,...,10,43,1,28,11,47,4,4,0,4


In [88]:
s = prod_df.dtypes == "float64"
for ind in s[s].index:
    prod_df.loc[:,ind] = prod_df.loc[:,ind].astype(str)

/tmp/ipykernel_106741/3800511137.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_df.loc[:,ind] = prod_df.loc[:,ind].astype(str)


In [89]:
prod_df

,status,name,okpd2,region,nmck,price,participans,bet,inn,0,...,date_month,date_week,date_weekend,date_day,date_hour,date_minute,date_dayofweek,date_quarter,is_holiday,date_season
0,1,СТУЛЬЯ УЧЕНИЧЕСКИЕ,0,2,596790.0,593806.05,1,1,2808,0.16128802299499512,...,6,26,1,30,11,20,2,2,0,4
1,0,МЕДИЦИНСКИЕ РАСХОДНЫЕ МАТЕРИАЛЫ,0,2,4964.9,0.0,0,0,443,0.19193466007709503,...,10,40,1,4,9,16,1,4,0,4
2,1,Видеокарта Palit PCI-E,0,2,462000.0,311850.0,6,65,56,0.22163449227809906,...,7,26,1,1,13,23,3,3,0,4
3,1,"ТОВАРЫ ИНФОРМАЦИОННО-ТЕХНОЛОГИЧЕСКИЕ, СРЕДСТВА...",0,2,505555.0,460042.9,6,18,2476,0.15981216728687286,...,3,11,1,16,10,42,1,1,0,4
4,1,Пинцет острый,0,2,2000.0,1980.0,2,2,2639,0.14485348761081696,...,5,21,1,26,11,4,3,2,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246757,0,Червячный хомут FORTISFLEX 110-130,0,2,1000.0,0.0,0,0,2484,0.19259922206401825,...,3,13,1,30,12,23,2,1,0,4
246758,0,"Маскировочная сеть, ""Стандарт"", МС1-6, 3х6 м",0,2,60000.0,0.0,0,0,2730,0.28064870834350586,...,10,40,1,3,9,42,0,4,0,4
246759,0,"ТРАНСПОРТНЫЕ СРЕДСТВА, РАСХОДНЫЕ МАТЕРИАЛЫ И З...",0,2,402582.6,0.0,0,0,1261,0.19415931403636932,...,4,16,1,20,9,13,1,2,0,4
246760,1,БЛАНКИ,0,2,18600.0,10500.0,5,9,2842,0.33276718854904175,...,10,43,1,28,11,47,4,4,0,4


In [71]:
prod_df = prod_df.iloc[:,~prod_df.columns.duplicated()]

# Set target and drop from df

In [90]:
X = prod_df.drop("status", axis=1)
y = prod_df["status"]

# Cat features

In [91]:
cat_features = list(range(0, X.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118]


In [92]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

In [93]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1, 
    #loss_function='CrossEntropy'
)


clf.fit(X_train, y_train, 
        cat_features=cat_features, 
        eval_set=(X_val, y_val), 
        verbose=False
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

CatBoost model is fitted: True
CatBoost model parameters:
{'iterations': 5, 'learning_rate': 0.1}


In [ ]:
from sklearn.metrics import Ma

In [98]:
catboost.metrics.MAE?

Init signature: catboost.metrics.MAE(**kwargs)
Docstring:     
Builtin metric: 'MAE'
Parameters:
    delta = 1e-06 (default value)
    alpha = 0.5 (default value)
    use_weights = True (default value)
    hints = '' (default value)
File:           ~/.venv/lib/python3.10/site-packages/catboost/metrics.py
Type:           _MetricGenerator
Subclasses:     


In [97]:
catboost.metrics.MAE(clf.predict(X_val), y_val)

TypeError: _MetricGenerator.__call__() takes 1 positional argument but 3 were given

In [ ]:
# import re


# for idx, row in enumerate(df_spgz):
#     kpgz = re.search(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', row).group()
#     print(kpgz)
#     if idx > 10:
#         break
    

In [ ]:
# """
# 1, 2, 3, 4, 5, 6 and 8 January — New Year Holidays;

# 7 January — Christmas Day;

# 23 February — Defender of the Fatherland Day;

# 8 March — International Women’s Day;

# 1 May — Spring and Labour Holiday;

# 9 May — Victory Day;

# 12 June — Day of Russia;

# 4 November — National Unity Day.
# """

# class RusHolidaysBusinessCalendar(AbstractHolidayCalendar):
#    rules = [
#      Holiday('New Year', month=12, day=31, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=1, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=2, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=3, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=4, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=5, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=6, observance=sunday_to_monday),
#      Holiday('New Year', month=1, day=8, observance=sunday_to_monday),
#      Holiday('8 March', month=3, day=8, observance=sunday_to_monday),
#      Holiday('1 May', month=5, day=1, observance=sunday_to_monday),
#      Holiday('9 May', month=5, day=9),
#      Holiday('12 June', month=6, day=12, observance=sunday_to_monday),
#      Holiday('4 November', month=5, day=1, observance=sunday_to_monday),
#      Holiday('11 November', month=11, day=11, observance=sunday_to_monday),
#    ]

# rus_BD = CustomBusinessDay(calendar=RusHolidaysBusinessCalendar())
# s = pd.date_range('2022-01-01', end='2022-12-31', freq=rus_BD)
# df = df.resample(rule=CustomBusinessMonthEnd(holidays=rus_BD)).last()
# df = pd.DataFrame(s, columns=['date'])
# print(df.tail())